## Deconvolutional Layers

Pulled and modified from: https://github.com/ardamavi/Cat-Segmentation

In [1]:
import os
import numpy as np
from keras.models import Model
from keras import backend as K
from keras.layers import Input, Conv2D, Activation, MaxPooling2D, Conv2DTranspose, concatenate

from os import listdir
from matplotlib import pyplot as plt
from scipy.misc import imread, imresize, toimage, imsave
from sklearn.model_selection import train_test_split

import sys
from get_dataset import get_img, save_img
from keras.models import model_from_json

from get_dataset import get_dataset
from get_model import get_model, save_model
from keras.callbacks import ModelCheckpoint, TensorBoard

from keras.utils import plot_model

Using TensorFlow backend.


------------------------- 
**Some helper functions are written below, we don't need to edit these**

In [2]:
"""
For the loss function, we will use the Sorensen-Dice coefficient which measures
the similarity between two images.
More information can be found here: 
https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient

"""

def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [3]:
"""
Code to retrieve images and save segmented data
"""


def get_img(data_path):
    # Getting image array from path:
    img = imread(data_path)
    img = imresize(img, (64, 64))
    return img

def save_img(img, name='segmentated.jpg'):
    imsave(name, img.reshape(64, 64))

def get_dataset(dataset_path='Data/Train_Data'):
    # Getting all data from data path:
    try:
        X = np.load('Data/npy_train_data/X.npy')
        Y = np.load('Data/npy_train_data/Y.npy')
    except:
        inputs_path = dataset_path+'/input'
        images = listdir(inputs_path) # Geting images
        X = []
        Y = []
        for img in images:
            img_path = inputs_path+'/'+img

            x_img = get_img(img_path).astype('float32').reshape(64, 64, 3)
            x_img /= 255.

            y_img = get_img(img_path.replace('input/', 'mask/mask_')).astype('float32').reshape(64, 64, 1)
            y_img /= 255.

            X.append(x_img)
            Y.append(y_img)
        X = np.array(X)
        Y = np.array(Y)
        # Create dateset:
        if not os.path.exists('Data/npy_train_data/'):
            os.makedirs('Data/npy_train_data/')
        np.save('Data/npy_train_data/X.npy', X)
        np.save('Data/npy_train_data/Y.npy', Y)
    X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
    return X, X_test, Y, Y_test

In [4]:
"""
Pre-built function to save the model's weights
"""
def save_model(model):
    if not os.path.exists('Data/Model/'):
        os.makedirs('Data/Model/')
    model_json = model.to_json()
    with open("Data/Model/model.json", "w") as model_file:
        model_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("Data/Model/weights.h5")
    print('Model and weights saved')
    return

-------------------------------
### Student code below.
We will build a basic conv to deconv network for image segmentation use. 

**Model Architecture, build the network like this:**   
  
Input Data Shape: 64x64x3
  
Convolutional Layer. 32 kernels with shape: 3x3. Strides: 1x1
  
Activation Function: ReLu
  
Convolutional Layer. 64 kernels with shape: 3x3. Strides: 1x1
  
Activation Function: ReLu
  
**Transpose Convolutional Layer. 64 kernels with shape: 3x3. Strides: 1x1**
  
Activation Function: ReLu
  
Merge Layer
  
**Transpose Convolutional Layer. 1 kernel with shape: 3x3. Strides: 1x1**
  
Activation Function: Sigmoid

In [5]:
#########################################################
####### Student code: modify this below #################
#########################################################

def get_model():
    
    ########### Images are 64x64 RGB
    inputs = Input(shape=(64, 64, 3))

    ####### First convolution layer
    conv_1 = Conv2D(1, (3, 3), strides=(1, 1), padding='same')(inputs)
    act_1 = Activation('relu')(conv_1)

    ###### Second convolution layer
    conv_2 = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(act_1)
    act_2 = Activation('relu')(conv_2)

######################################################

#Edits go here
#First deconv layer. Use keras Conv2DTranspose with similar inputs to the Conv2D layer

#deconv_1 = Conv2DTranspose(filters, kernel_size, strides=(1, 1), padding='valid' or 'same')

    
    act_3 = Activation('relu')(deconv_1)
######################################################
    
    ##### Now merge layer 1 with layer 3
    merge_1 = concatenate([act_3, act_1], axis=3)
    
###################################################

#Second edits go here
#Second Deconv layer, Use keras Conv2DTranspose with similar inputs to the Conv2D layer
#deconv_2 = Conv2DTranspose(filters, kernel_size, strides=(1, 1), padding='valid' or 'same')
#this one is after the merge layer, so be sure to use (merge_1)

    
    
    act_4 = Activation('relu')(deconv_2)
##################################################
    
    ##### Compile the model
    model = Model(inputs=[inputs], outputs=[act_4])
    model.compile(optimizer='adadelta', loss=dice_coef_loss, metrics=[dice_coef])

    return model

if __name__ == '__main__':
     save_model(get_model())

Instructions for updating:
Colocations handled automatically by placer.
Model and weights saved


![title](ConvDeconv.png)

**Now train the model for 50 epochs:**

In [6]:
##############################################
################ change this to True to train!
traincats = False
############################################

epochs = 50
batch_size = 5

if traincats == True:

    def train_model(model, X, X_test, Y, Y_test):
        if not os.path.exists('Data/Checkpoints/'):
            os.makedirs('Data/Checkpoints/')
        checkpoints = []
        checkpoints.append(ModelCheckpoint('Data/Checkpoints/best_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1))
        checkpoints.append(TensorBoard(log_dir='Data/Checkpoints/./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None))
    
        model.fit(X, Y, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test), shuffle=True, callbacks=checkpoints)
    
        return model
    
    def main():
        X, X_test, Y, Y_test = get_dataset()
        model = get_model()
        model = train_model(model, X, X_test, Y, Y_test)
        save_model(model)
        
        return model
    
    if __name__ == '__main__':
        main()

Train on 72 samples, validate on 9 samples
Epoch 1/50
72/72 [==============================] - 1s 19ms/step - loss: -0.4574 - dice_coef: 0.4574 - val_loss: -0.7484 - val_dice_coef: 0.7484
Epoch 2/50
72/72 [==============================] - 1s 17ms/step - loss: -0.8475 - dice_coef: 0.8475 - val_loss: -0.7795 - val_dice_coef: 0.7795
Epoch 3/50
72/72 [==============================] - 1s 17ms/step - loss: -0.8759 - dice_coef: 0.8759 - val_loss: -0.7883 - val_dice_coef: 0.7883
Epoch 4/50
72/72 [==============================] - 1s 17ms/step - loss: -0.8788 - dice_coef: 0.8788 - val_loss: -0.7915 - val_dice_coef: 0.7915
Epoch 5/50
72/72 [==============================] - 1s 17ms/step - loss: -0.8831 - dice_coef: 0.8831 - val_loss: -0.7938 - val_dice_coef: 0.7938
Epoch 6/50
72/72 [==============================] - 1s 17ms/step - loss: -0.8836 - dice_coef: 0.8836 - val_loss: -0.7946 - val_dice_coef: 0.7946
Epoch 7/50
72/72 [==============================] - 1s 17ms/step - loss: -0.8840 - dice

**Now pick a cat to segment:**

In [7]:
"""
Go into Data/Train_Data/input/ and pick your favorite cat image.
Then segment it by changing the number in cat.##.jpg below to the one you want.
Try a few different ones and compare outputs!
"""

if traincats == True:
    %run -i "predict.py" "Data/Train_Data/input/cat.39.jpg"

Segmentated image saved as segmentated.jpg
